### **EVALUACION DE 4 MODIFICACIONES A LA ARQUITECTURA # 3**



### **Exploración de datos**

Cargamos todos los recursos y procesamos las imagenes. Estos procesos estan descritos a detalle en el archivo ***01 - Exploración y preprocesado de datos.ipynb***

In [ ]:
!pip install gitpython
import git
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# URL del repositorio de GitHub
repo_url = 'https://github.com/keonij1/Tomato-Leaf-Disease-Detection'
# Directorio de destino para clonar el repositorio
destination_folder = '/Data'
# Verificar si el directorio de destino ya existe
if not os.path.exists(destination_folder):
    # Clonar el repositorio si el directorio no existe
    git.Repo.clone_from(repo_url, destination_folder)
else:
    print("El repositorio ya ha sido clonado anteriormente.")

# Directorio donde se clonó el repositorio
destination_folder = '/Data'
repo_dir = os.path.join(destination_folder, 'Data')
# Directorio de imágenes
images_dir = os.path.join(repo_dir, 'train')
# Directorio raíz que contiene las 10 carpetas
datos_graficar = images_dir
# Obtener la lista de carpetas dentro del directorio raíz
carpetas = os.listdir(datos_graficar)
EPOCHS = 25  # Número de épocas para entrenamiento del modelo
INIT_LR = 1e-3  # Tasa de aprendizaje inicial del modelo
BS = 32  # Tamaño del lote (batch size)
default_image_size = tuple((256, 256))  # Tamaño predeterminado de las imágenes (ancho, alto)
image_size = 0  # Tamaño de las imágenes utilizadas en el proceso
directory_root = repo_dir  # Ruta del directorio raíz donde se encuentran los datos o imágenes
width = 256  # Ancho de las imágenes
height = 256  # Alto de las imágenes
depth = 3  # Profundidad de las imágenes (número de canales, en este caso 3 para RGB)

def convert_image_to_array(image_dir):
    try:
        # Lee la imagen utilizando cv2.imread()
        image = cv2.imread(image_dir)
        # Verifica si la imagen se ha leído correctamente
        if image is not None:
            # Redimensiona la imagen a un tamaño específico
            image = cv2.resize(image, default_image_size)
            # Convierte la imagen en una matriz numpy utilizando img_to_array()
            return img_to_array(image)
        else:
            # Devuelve una matriz vacía si la imagen no se ha leído correctamente
            return np.array([])
    except Exception as e:
        # Captura cualquier excepción que ocurra y muestra un mensaje de error
        print(f"Error: {e}")
        # Devuelve None en caso de error
        return None

image_list, label_list = [], []  # Listas vacías para almacenar las imágenes y las etiquetas correspondientes
try:
    root_dir = listdir(directory_root)  # Obtener una lista de los directorios en el directorio raíz
    for directory in root_dir:
        if directory == ".DS_Store":  # Eliminar el archivo .DS_Store si está presente
            root_dir.remove(directory)
    for plant_folder in root_dir:
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")  # Obtener una lista de las carpetas de enfermedades de las plantas en el directorio de la planta actual
        for disease_folder in plant_disease_folder_list:
            if disease_folder == ".DS_Store":  # Eliminar el archivo .DS_Store si está presente
                plant_disease_folder_list.remove(disease_folder)
        for plant_disease_folder in plant_disease_folder_list:
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")  # Obtener una lista de las imágenes en la carpeta de enfermedades de plantas actual
            for single_plant_disease_image in plant_disease_image_list:
                if single_plant_disease_image == ".DS_Store":  # Eliminar el archivo .DS_Store si está presente
                    plant_disease_image_list.remove(single_plant_disease_image)
            for image in plant_disease_image_list[:200]:  # Iterar sobre las primeras 200 imágenes de la lista de imágenes de enfermedades de plantas
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"  # Directorio de la imagen actual
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:  # Verificar si el archivo es una imagen JPG
                    image_list.append(convert_image_to_array(image_directory))  # Convertir la imagen a un array y agregarla a la lista de imágenes
                    label_list.append(plant_disease_folder)  # Agregar la etiqueta de la enfermedad de la planta a la lista de etiquetas
except Exception as e:
    print(f"Error : {e}")

image_size = len(image_list)  # Tamaño de la lista de imágenes
label_binarizer = LabelBinarizer()  # Inicializar un objeto LabelBinarizer para codificar las etiquetas
image_labels = label_binarizer.fit_transform(label_list)  # Codificar las etiquetas de las imágenes
pickle.dump(label_binarizer, open('label_transform.pkl', 'wb'))  # Guardar el objeto LabelBinarizer en un archivo
n_classes = len(label_binarizer.classes_)  # Número de clases en el conjunto de datos
np_image_list = np.array(image_list, dtype=np.float16) / 225.0  # Convierte la lista de imágenes en un arreglo de NumPy y normaliza los valores de píxeles entre 0 y 1
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state=42)  # Divide los datos en conjuntos de entrenamiento y prueba

aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2,
    zoom_range=0.2,horizontal_flip=True,
    fill_mode="nearest")

print("Los datos han sido procesados exitosamentes y esta estan listos para ser usados!")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
El repositorio ya ha sido clonado anteriormente.
Los datos han sido procesados exitosamentes y esta estan listos para ser usados


###  **ARQUITECTURA 3**

Sobre esta arquitectura se realizaran 4 iteraciones ,en las cuales se realizaran diferentes modificaciones a la arquitectura


In [ ]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1

model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))


# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 activation (Activation)     (None, 256, 256, 16)      0         
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 16)     64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 16)      2320      
                                                                 
 activation_1 (Activation)   (None, 256, 256, 16)      0         
                                                                 
 batch_normalization_1 (Batc  (None, 256, 256, 16)     64        
 hNormalization)                                        

###  **ITERACION # 1**

En esta iteracion se propone aumentar el número de filtros en las primeras 3 capas convolucionales. En este caso se duplicara el número de filtro

In [ ]:
# Crear el modelo
model = Sequential()

inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1

model.add(Conv2D(64, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.plot(epochs, acc, 'bo-', label='Training accuracy', markersize=4)
plt.plot(epochs, val_acc, 'r*-', label='Validation accuracy', markersize=4)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epocas')
plt.ylabel('Precisión')
plt.legend(loc='lower right')
plt.grid(True, linestyle='--')

# Annotate the points with their values for accuracy
for ep, vac in zip(epochs, val_acc):
    plt.annotate(f'{vac:.3f}', xy=(ep, vac), xytext=(ep, vac - 0.04), ha='center', color='r')

# Train and validation loss
plt.figure()  # Create a new figure for loss plot
plt.plot(epochs, loss, 'bo-', label='Training loss', markersize=4)
plt.plot(epochs, val_loss, 'r*-', label='Validation loss', markersize=4)
plt.title('Training and Validation Loss')
plt.xlabel('Epocas')
plt.ylabel('Pérdida')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--')

#plt.tight_layout()
plt.show()

print("Calculando la precision")
scores = model.evaluate(x_test, y_test)
print(f"Precisión en la validacion: {scores[1]*100}")

###  **ITERACION # 2**

En esta iteracion se propone cambiar el tamaño del kernel en las primeras 3 capas convolucionales. En es te caso se usara (5, 5) en lugar de (3, 3)

In [ ]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1

model.add(Conv2D(32, (5, 5), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(Conv2D(32, (5, 5), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(32, (5, 5), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))


# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 256, 256, 32)      896       
                                                                 
 activation_8 (Activation)   (None, 256, 256, 32)      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 256, 256, 32)      9248      
                                                                 
 activation_9 (Activation)   (None, 256, 256, 32)      0         
                                                                 
 conv2d_8 (Conv2D)           (None, 256, 256, 32)      9248      
                                                                 
 activation_10 (Activation)  (None, 256, 256, 32)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 128, 128, 32)    

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.plot(epochs, acc, 'bo-', label='Training accuracy', markersize=4)
plt.plot(epochs, val_acc, 'r*-', label='Validation accuracy', markersize=4)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epocas')
plt.ylabel('Precisión')
plt.legend(loc='lower right')
plt.grid(True, linestyle='--')

# Annotate the points with their values for accuracy
for ep, vac in zip(epochs, val_acc):
    plt.annotate(f'{vac:.3f}', xy=(ep, vac), xytext=(ep, vac - 0.04), ha='center', color='r')

# Train and validation loss
plt.figure()  # Create a new figure for loss plot
plt.plot(epochs, loss, 'bo-', label='Training loss', markersize=4)
plt.plot(epochs, val_loss, 'r*-', label='Validation loss', markersize=4)
plt.title('Training and Validation Loss')
plt.xlabel('Epocas')
plt.ylabel('Pérdida')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--')

#plt.tight_layout()
plt.show()

print("Calculando la precision")
scores = model.evaluate(x_test, y_test)
print(f"Precisión en la validacion: {scores[1]*100}")

###  **ITERACION # 3**

En esta iteracion se propone agregar capas de normalización por lotes (Batch Normalization) después de las capas de convolución. Esto con el fin de intentar ayudar a estabilizar el entrenamiento y acelerar la convergencia del modelo.


In [ ]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1

model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
model.summary()

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.plot(epochs, acc, 'bo-', label='Training accuracy', markersize=4)
plt.plot(epochs, val_acc, 'r*-', label='Validation accuracy', markersize=4)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epocas')
plt.ylabel('Precisión')
plt.legend(loc='lower right')
plt.grid(True, linestyle='--')

# Annotate the points with their values for accuracy
for ep, vac in zip(epochs, val_acc):
    plt.annotate(f'{vac:.3f}', xy=(ep, vac), xytext=(ep, vac - 0.04), ha='center', color='r')

# Train and validation loss
plt.figure()  # Create a new figure for loss plot
plt.plot(epochs, loss, 'bo-', label='Training loss', markersize=4)
plt.plot(epochs, val_loss, 'r*-', label='Validation loss', markersize=4)
plt.title('Training and Validation Loss')
plt.xlabel('Epocas')
plt.ylabel('Pérdida')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--')

#plt.tight_layout()
plt.show()

print("Calculando la precision")
scores = model.evaluate(x_test, y_test)
print(f"Precisión en la validacion: {scores[1]*100}")

###  **ITERACION # 4**

En esta iteracion se propone agregar una capa de regularización L2 (Weight Decay) a las capas totalmente conectadas (Dense) para penalizar los valores de los pesos grandes y reducir el sobreajuste

In [ ]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1


model.add(Conv2D(64, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(n_classes, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation("softmax"))



# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mostrar el resumen del modelo
model.summary()

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.plot(epochs, acc, 'bo-', label='Training accuracy', markersize=4)
plt.plot(epochs, val_acc, 'r*-', label='Validation accuracy', markersize=4)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epocas')
plt.ylabel('Precisión')
plt.legend(loc='lower right')
plt.grid(True, linestyle='--')

# Annotate the points with their values for accuracy
for ep, vac in zip(epochs, val_acc):
    plt.annotate(f'{vac:.3f}', xy=(ep, vac), xytext=(ep, vac - 0.04), ha='center', color='r')

# Train and validation loss
plt.figure()  # Create a new figure for loss plot
plt.plot(epochs, loss, 'bo-', label='Training loss', markersize=4)
plt.plot(epochs, val_loss, 'r*-', label='Validation loss', markersize=4)
plt.title('Training and Validation Loss')
plt.xlabel('Epocas')
plt.ylabel('Pérdida')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--')

#plt.tight_layout()
plt.show()

print("Calculando la precision")
scores = model.evaluate(x_test, y_test)
print(f"Precisión en la validacion: {scores[1]*100}")